In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.exceptions import InvalidSignature

In [2]:
def generate_private_key():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    return private_key


def get_private_pem(private_key):
    pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    return pem


def get_public_pem(public_key):
    pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    return pem


def get_message_signature(message, private_key):
    signature = private_key.sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return signature


def is_valid_signature(signature, message, public_key):
    try:
        public_key.verify(
            signature,
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except InvalidSignature as e:
        return False
    
    
def cipher(message, public_key):
    ciphertext = public_key.encrypt(
        message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext


def decipher(ciphertext, private_key):
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return plaintext

## create keypairs for the first and the second persons

In [3]:
keys = [{} for x in range(2)]

keys[0]['private'] = generate_private_key()
keys[0]['public'] = keys[0]['private'].public_key()

keys[1]['private'] = generate_private_key()
keys[1]['public'] = keys[1]['private'].public_key()

## sign the message as the first person

In [4]:
message = b"A message I want to sign.\nNew line.\tTab\tTab"
print(message.decode())

A message I want to sign.
New line.	Tab	Tab


In [5]:
signature = get_message_signature(message, keys[0]['private'])
signature

b'\xd1;\xf7\xfb\xfdf\xd2\r\x96O\x0f\x00\xc1\x02\xba\xff\xde\xc3:\xcc\x04\xfd"\xa4\xd4\xa7\xbeP\xc5\xd1S\xf0\xd2\xa0\xd4\xebn\xba\xa7\xaf\xa1\\\x01\x1d\xdb\x83\xb5\xf9.!\xf0"v\xa77\xd0\t\x198\x04\xfcPq%\x05b"Ur\x0f\xd3\x8b9\xd4\x17\xf2GI\xfcM\x1a\xe3\x19w\xaeF\xd1\\6:\x02apQ\xd5Q\xef;\xc8\x1c\x01,\xa2\x87\x91h\xa6\x04\xad9N\x95\x9d\xc8\x1a\xc1Dv\xf0\n\xc1\xab\x16M\xf7\xb0\x0e\t\xe1\xdd"\xe2\x1e\xbbYm\xc3\n\x17&\xbe\xe6\xec\'\xaem\xf2\xcb\x9d\xd7\xfbR\xa3h\x7f^\xb3R\x10\x18\xc1\r\x13,\xff4\x80\x95;3S\xa9i\xb2;\x19\x85SP\x9d\xfa\xc67<\x1c~\xd1:W\xb3\x80\x8e\xb1D\xfa\x91\x00\xd6&\xbb\xe3X\xb2\\[E\xab\x82\xeb\xa6\xbb;\xc7\x80fIi\xca\xc1\xf8\x8d\x05\x8f\xf0\xeer\x82\x16\x87C\xc8\xcd\xa9\xda\xd0\xc7\x9dyt18\x13\xdav\x90\xe1vI\xd4\\\xf9\xbc\xc5\xe0On'

In [6]:
is_valid_signature(signature, message, keys[0]['public'])

True

In [7]:
is_valid_signature(signature, message, keys[1]['public'])

False

## encode message for the second person

In [8]:
message = b"Some valuable\t\tinformation\nto be transmitted!!\n\nEnd of the message"
print(message.decode())

Some valuable		information
to be transmitted!!

End of the message


In [9]:
ciphertext = cipher(message, keys[1]['public'])
ciphertext

b'R\x11\x7fN\xbb\x0b0\x7fR\xc0.\xa1(\x94\xdbc\x19h\xdaIj\xb8S%\xef{\x18\xf0\xc9\xc2\x99\x8a\xbc\xe0\x85\xe9\x023\xdaS\xe7@Y\x92\x0eo\xcc\xe5v\xf8km\xa0\x08\x99H=\x9e7\x9b8(\xc4\xaf\xdf~\x10a\xf1\x0e\xa1\xc1\xd5i\x9e\xe0\xdc\xa9\x16_\x81\xda\xfc\x7f[\xcdb\x172Y\xdf\xca-l\x8b\xe8?\xca\xdc\xc3\xb6\xee~|\xda\x15\x94\x97\xd5\xbe\xeaJ\xe1(\xbf\x84\xa6[Z\x0b\x9c\x7f\x1e\xb9Rvx\xab\x8b\xd3M\x05@=\x02\xa0\xaa\x99\xb6\x85\xf2\x0bv\xdcxW}\x8a\xc2G\xff>2\x9a\xe3\xc5\x88C\x12\x91W\x07!\x07\x91@\xab{\xc6\x84\x1d\x8a\xc1\xe7\x8eRE\xf08\xfa*Ez\x9fP\xa9\x0b\xf0\xd0\xdb\x06K\x90\x8c\xfc\x10\xfb\x02`\xc8ux\x8a\x9f\xdc\xb3\xddd\xeb2d\xaa\xbc\xe5D\x12Y\xff\xd5Bo\x86\xdd\xbfn\x8e\xa6MN\x07\x1d\xa6\x10x\x8c8\xe0Z\x85\xa7(S\xcb\xe2~\xe5\x9as\x08E\x10@4\x85\x94\x98'

In [10]:
plaintext = decipher(ciphertext, keys[1]['private'])
print(plaintext)

b'Some valuable\t\tinformation\nto be transmitted!!\n\nEnd of the message'


In [11]:
print(plaintext == message)

True
